<a href="https://colab.research.google.com/github/ZahraDehghani99/ML_MSc/blob/main/HW5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Work 5-3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## EDA

In [ ]:
#load the preprocessing dataframe
data = pd.read_pickle(r'/content/drive/MyDrive/ML/HW5/data_preprocessed')
data.head()

,Age,WorkClass,FinancialWeight,Education,Education-num,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HourPerWeek,NativeCountry,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Let's check for missing values.

In [ ]:
print(f'number of missing values : {data.isnull().sum().sum()}')

number of missing values : 0


In [ ]:
data.nunique() # Get unique count for each variable

Age                   73
WorkClass              8
FinancialWeight    21648
Education             16
Education-num         16
MaritalStatus          7
Occupation            14
Relationship           6
Race                   5
Sex                    2
CapitalGain          119
CapitalLoss           92
HourPerWeek           94
NativeCountry         41
Income                 2
dtype: int64

### Drop Redundant features

As we can see in the above cell, `Education` and `Education-num` are equivalent. we should drop one of them to not have redundant features. we drop `Education` column.


In [ ]:
data.drop('Education', inplace=True, axis=1)
print(f'data shape : {data.shape}')

data shape : (32561, 14)


In [ ]:
data.head()

,Age,WorkClass,FinancialWeight,Education-num,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HourPerWeek,NativeCountry,Income
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Split X and y

In [ ]:
X = data.drop(columns='Income')
y = pd.DataFrame(data['Income'])

print(f'X shape : {X.shape}')
print(f'y shape : {y.shape}')

X shape : (32561, 13)
y shape : (32561, 1)


### Encoding Categorical features

Let's encode categorical features using OrdinalEncoder in sklearn. We should use these features in CategoricalNB and using GaussianNB with the rest of the features.

In [ ]:
enc = OrdinalEncoder()
X_C = X[['WorkClass', 'MaritalStatus', 'Occupation', 'Relationship', 'Race', 'Sex', 'NativeCountry']]
X_C = enc.fit_transform(X_C)

In [ ]:
print(f'X_C shape : {X_C.shape}')

X_C shape : (32561, 7)


In [ ]:
X_C

array([[ 6.,  4.,  0., ...,  4.,  1., 38.],
       [ 5.,  2.,  3., ...,  4.,  1., 38.],
       [ 3.,  0.,  5., ...,  4.,  1., 38.],
       ...,
       [ 3.,  6.,  0., ...,  4.,  0., 38.],
       [ 3.,  4.,  0., ...,  4.,  1., 38.],
       [ 4.,  2.,  3., ...,  4.,  0., 38.]])

let's encoding label of classes.


In [ ]:
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
print(f'y shape after label encoding: {y.shape}')

y shape after label encoding: (32561,)


In [ ]:
print(f'{le.inverse_transform([0, 1])[0]} mapped to 0 and {le.inverse_transform([0, 1])[1]} mapped to 1')

 <=50K mapped to 0 and  >50K mapped to 1


## Classification using hybrid Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.model_selection import cross_val_score

In [ ]:
# Split numerical features
X_G = X[['Age', 'FinancialWeight', 'Education-num', 'CapitalGain', 'CapitalLoss', 'HourPerWeek']] # Gaussian, i.e. continuous

# Combine all four variables into one array (concatenation along the second axis)
X = np.c_[X_G, X_C[:,0].ravel(), X_C[:,1].ravel(), X_C[:,2].ravel(), X_C[:,3].ravel(), X_C[:,4].ravel(),X_C[:,5].ravel(), X_C[:,6].ravel()]
print(f'X shape: {X.shape}')

X shape: (32561, 13)


In [ ]:
# let's Fit the two models
# Now use the Gaussian model for continuous independent variable and 
model_G = GaussianNB()
clf_G = model_G.fit(X[:,0:7], y)
# Categorical model for discrete independent variable
model_C = CategoricalNB()
clf_C = model_C.fit(X[:,7:-1], y)


# Get probability predictions from each model 
G_probas = model_G.predict_proba(X[:,0:7])
C_probas = model_C.predict_proba(X[:,7:-1])


# Combine probability prediction for class=1 from both models into a 2D array
X_new = np.c_[(G_probas[:,1], C_probas[:,1])]

# Fit Gaussian model on the X_new 
model = GaussianNB()
scores = cross_val_score(model, X_new, y, cv=10)
print(f'accuracy of hybrid :{scores.mean()}')


accuracy of hybrid :0.7953687446203566
